# Reinforcement Learning Example

This notebook will guide you through implementing a gradient policy algorithm in neon to train an agent to play Pong. You will learn
- Using the Autodiff interface to perform automatic differentiation and obtain gradients given an op-tree
- Implement a neural network that adjusts its parameters and policies with gradients to get better rewards directly from experiences. This model samples the expected policy from a distribution. 
  

## Preamble

Reinforcement Learning trains an agent to take actions in an unknown environment based on experiences. Unlike supervised learning, actual labels are not always available so RL algorithms explores the environment by interacting with actions and receives corresponding feedbacks as positive or negative rewards. 

In this tutorial, we will train an agent to play Pong with Stochastic Policy Gradients using [OpenAI Gym](https://gym.openai.com/) as the environment and [neon](https://neon.nervanasys.com/index.html/) as our deep learning framework. 

![Agent interacting with its environment](data/pong_rl_demo.gif)

As described in [Andrei Karpathy's blog](http://karpathy.github.io/2016/05/31/rl/) Policy Gradients has shown to perform better than DQN algorithms by most people including DQN Authors (https://www.youtube.com/watch?v=M8RfOCYIL8k), so we will use it to 

The underlying model is a 2-layer fully connected neural network that receives image frames as inputs, explores rewards given stochastic actions (probability of moving a paddle UP/DOWN) in an unknown environment, and updates weights by maximizing a temporal cumulative reward function modulated by an advantage as depicted below.  

![RL algorithm](data/pong_architecture.jpg)


## Setup

This example works with Python 2.7. 

Your environment needs to have the following packages installed:
- neon v1.9.0
- OpenAI Gym for initializing Atari game environments
- numpy for random initialization and aritmetic operations


# Model Architecture

We will guide you through implementing a policy function parameterized by a neural network. We first import all the needed ingredients: 

In [ ]:
import numpy as np
import gym
from neon.backends import gen_backend
from neon.backends import Autodiff

We also set up the backend and define the class containing our network. This consists of two layers 'W1' and 'W2' with values randomly initialized, but trainable with gradient updates. 

We have included functions implementing the forward and backward propagation steps. The forward step generates a policy given an a visual representation of the environment stored in variable x. The back propagation function updates the layers of the network modulating the loss function values with discounted rewards.

In [ ]:
be = gen_backend('cpu', batch_size=128)
class Network:
    def __init__(self, D = 80*80, H = 200, gamma = 0.99):
        '''
        D: number image pixels
        H: number of hidden units in first layer of neural network
        gamma: discount factor
        '''
        self.gamma = gamma
        self.ll = {}
        self.learning_rate = 0.001

        self.ll['W1'] = be.array(np.random.randn(H,D) / np.sqrt(D)) #be.zeros((H,D))
        self.ll['W2'] = be.array(np.random.randn(H,1) / np.sqrt(H)) #be.zeros((H,1))

    # forward propagation
    def policy_forward(self, x):
        # map visual input to the first hidden layer of a neural network
        # a larger number of units will increase the capacity of the network to learn different game states
        # different local minima in this context represents different strategies giving the same game output
        h = be.dot(self.ll['W1'],  be.array(x))
        h = be.tanh(h)
        logp = be.dot(h.transpose(), self.ll['W2'])

        #probability of moving paddle up and hidden state
        p = be.sig(logp)

        #execute:
        p_val = be.empty((1, 1))
        h_val = be.empty((200, 1))
        p_val[:] = p
        h_val[:] = h

        return p_val.get(), h_val.get(), p, h

    # backward propagation
    def policy_backward(self, losses_op, episode_losses, episode_rewards):
        discounted_rewards = self.discount_rewards(episode_rewards)
        # to reduce the variance of the gradient estimator and avoid potential vanishing problems
        # when computing gradients: http://www.scholarpedia.org/article/Policy_gradient_methods
        discounted_rewards -= np.mean(discounted_rewards)
        discounted_rewards /= np.std(discounted_rewards)

        episode_losses *= discounted_rewards  # to modulate gradients with discount factors

        #compute partial derivatives using neon backend
        for i in range(len(losses_op)):
            ad = Autodiff(op_tree=losses_op[i] * be.array(discounted_rewards[i]), be=be, next_error=None)
            # gradients
            dW1, dW2 = ad.get_grad_asnumpyarray([self.ll['W1'], self.ll['W2']])
            # execute:
            dW1_val = be.empty((200, 6400))
            dW2_val = be.empty((200, 1))
            dW1_val[:] = dW1
            dW2_val[:] = dW2
            self.ll['W2'][:] = self.ll['W2'].get() + self.learning_rate * dW2_val.get()
            self.ll['W1'][:] = self.ll['W1'].get() + self.learning_rate * dW1_val.get()
        return

    def sigmoid(self, x): 
        return 1.0 / (1.0 + np.exp(-x)) # sigmoid "squashing" function to interval [0,1]

    def get_loss(self, y, up_probability):
        loss = y-up_probability
        return loss
    
    # which actions lead to winning a game?: given this episodic environment, we assign rewards per time-step assuming
    # there is a stationary distribution for a policy within the episode.
    #   reward < 0, if agent missed the ball and hence lose the game
    #   reward > 0, if agent won the game
    #   reward == zero, any other state during the duration of a game
    # the agent receives rewards generated by the game and implements discounted reward backwards with exponential
    # moving average. More weight is given to earlier rewards. Reset to zero when game ends.
    def discount_rewards(self, r):
      discounted_r = np.zeros_like(r)
      for t in reversed(range(0, r.size)):
        # if reward at index t is nonzero, then there is a positive/negative reward. This also marks a game boundary
        # for the sequence of actions produced by the agent
        if r[t] != 0: running_add = 0 
        # moving average given discount factor gamma, it assigns more weight on recent actions
        discounted_r[t] = r[t] + discounted_r[t] * self.gamma
      return discounted_r

    # takes a single game frame as input and preprocesses before feeding into model
    def prepro(self, I):
      """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
      I = I[35:195] # crop
      I = I[::2,::2,0]              # downsample by factor of 2
      I[I == 144] = 0               # erase background (background type 1)
      I[I == 109] = 0               # erase background (background type 2)
      I[I != 0] = 1                 # everything else (paddles, ball) just set to 1
      return I.astype(np.float).ravel() #flattens



Part of the magic behind reinforcement learning is related to the notion of optimizing objective functions under uncertain or non-differentiable scenarios considering a stochastic process. In our case, we sample the action from the probabilities returned by the last layer of a neural network and consider it as a temporal label. This logic is implemented in the following function.  

In [ ]:
# stochastic process to choose an action (moving up) proportional to its
# predicted probability. If such value is high, it's more
# likely to sample an up action from this stochastic process.
# Probablity of choosing the opposite action is (1-probability_up)
#    action_ == 2, moving up
#    action_ == 3, moving down
def sample_action(up_probability):
    stochastic_value = np.random.uniform()
    action = 2 if stochastic_value < up_probability else 3
    return action

Initialization of variables.

In [ ]:
D = 80 * 80                 # number of pixels in input
H = 200                     # number of hidden layer neurons
# Pong environment
env = gym.make("Pong-v0")
network = Network(D=D, H=H)

# Each time step, the agent chooses an action, and the environment returns an observation 
# and a reward. The process gets started by calling reset, which returns an initial observation
observation = env.reset()
prev_x = None

# hidden state, gradient ops, gradient values, rewards
hs, losses_op, losses_values, rewards, h_values, x_values = [],[],[], [], [], []
running_reward = None       # current reward
reward_sum = 0.0            # sum rewards
episode_number = 0

## Let's train!

Our goal is to train an agent to win Pong against its opponent. An action consists of moving a paddle UP/DOWN, this eventually generates a positive reward (+1) if the trained agent wins a game or negative one (-1) if agent misses the ball.

Before knowing the result of a game, the model gets a fake label via the stochastic process explained before. This is like tossing a coin to decide to accept the log probabilities of a neural network. An optimal set of actions will maximize the sum of rewards along the game. An import event is when the agent wins/losses a game. But what caused this outcome?. The algorithm decided to modulate the loss functions of the network with the positive or negative rewards obtained from the environment and assign more weight to earlier actions using a moving average scheme. This logic is implement in function policy_backward() of the Network class.


In [ ]:
while True:
    cur_x = network.prepro(observation)
    x = cur_x - prev_x if prev_x is not None else np.zeros(D)
    prev_x = cur_x

    up_probability, h_value, p, h = network.policy_forward(x)
    action = sample_action(up_probability)

    y = 1 if action == 2 else 0                 # assign a fake label, this decreases uncertainty and
    loss_value = y - up_probability             # this is one of the beauties of Reinforcement Learning
    losses_values.append(loss_value)
    losses_op.append(y - p)
    h_values.append(h_value)
    env.render()
    observation, reward, done, info = env.step(action)
    rewards.append(reward)
    reward_sum += reward
    x_values.append(x)

    if done:
        episode_number +=1
        episode_losses = np.vstack(losses_values)
        episode_rewards = np.vstack(rewards)
        losses_values, rewards, h_values = [], [], []

        network.policy_backward(losses_op, episode_losses, episode_rewards)

        losses_op = []
        x_values = []

        mean_loss = np.sum([x * x for x in episode_losses])
        running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01

        print('-----------------------------------------------')
        print('Episode %d has finished, time to backpropagate.' % (episode_number - 1))
        print('Total reward was %f Running_reward: %f Mean_loss: %f' % (reward_sum, running_reward, mean_loss))
        print('-----------------------------------------------')

        reward_sum = 0
        observation = env.reset()  # reset env
        prev_x = None

    if reward != 0:  # Pong has either +1 or -1 reward exactly when game ends.
        message = "Episode %d: game finished. Reward: %f Loss: %f\t" % (episode_number, reward, loss_value)
        if reward == -1:
            message += "\x1b[0;31;40m  (RL loses)\x1b[0m"
        else:
            message += "\x1b[0;32;40m  (RL wins)\x1b[0m"
        print(message)


## Policy Gradients

Consider the below visualization to get an intuition about how policy gradients behave in reinforcement learning. We map to 2D the weights 'W2' of the network before and after policy backpropagation takes place after 100 episodes. 

Left plot: The points in green correspond to observations associated to positive rewards and the ones in red are samples producing negative or zero rewards. Note how unbalanced is this problem as positive rewards are by definition sparse.   

Right plot: The arrows represent the gradient of each sample towards the direction that increases its expected reward. Since the loss values of the network are weighted by their standardized  cumulative rewards in an episode, we expect the mean of a distribution to be influenced by the few gradients with positive rewards after parameter updates. Indeed, regions around green points seem to have small gradients indicating the preference of the network to keep those weight values. On the contrary, other regions often have large gradients which means back propagation algorithm notably updates their values in seek of better regions for maximizing the overall reward. After parameter update, the distribution will slowly move towards points with positive rewards. This is a slow process and the reason why reinforcement learning algorithms needs lots of training time. 

![RL algorithm](data/gradient_update.png)


Other episodes.

Episode 115:
![RL algorithm](data/gradient_update-115.png)

Episode 135:
![RL algorithm](data/gradient_update-135.png)